In [1]:
import pandas as pd
import geopandas as gpd
import json as js
import matplotlib.pyplot as plt
import re
import numpy as np
import folium
from folium import IFrame
from shapely.geometry import Polygon
from bs4 import BeautifulSoup
from pykml import parser

In [4]:
fuel_df = pd.read_csv('galvanize/MIDTERM /National_Alt_Fuel_Database/data/alt_fuel_stations (Jul 29 2021).csv')
fuel_df.head()

/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_21513/2585286318.py:1: DtypeWarning: Columns (3,6,10,13,14,15,16,20,29,31,33,34,35,36,43,45,46,47,49,52,53,54,56,57,58,59,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  fuel_df = pd.read_csv('galvanize/MIDTERM /National_Alt_Fuel_Database/data/alt_fuel_stations (Jul 29 2021).csv')


,Fuel Type Cotion Directions,City,State,ZIP,Plus4,Station Phone,Status Code,Expected Date,Groups With Access Code,Access Days Time,...,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Accessde,Station Name,Street Address,Intersec
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,MD,NaN,NaN,NaN
1,CNG,PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Ave, left on...",Atlanta,GA,30303,NaN,770-350-3000,E,...,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN,NaN
2,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324,NaN,NaN,E,...,NaN,NaN,NaN,NaN,Q,3000,LD,NaN,NaN,NaN
3,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,HD,NaN,NaN,NaN
4,CNG,Clean Energy - Texas Department of Transportation,7721A Washington St,"I-10, Washington Ave exit, 1.5 blocks to the s...",Houston,TX,77007,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,Q,3000 3600,MD,NaN,NaN,NaN
